In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pickle

import sys
sys.path.append('../../')
from Pulses import STA_pulses
from QEnvs.QEnvWave import QEnvWave
from Noises import Noise
from Plots import plot_noise_effect

2023-06-20 18:38:36.518853: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-20 18:38:37.185154: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
episode = np.load("Data/3/3_n_deltas/eval_best_episode.npy", allow_pickle=True)
me = False
noise_type = "gaussian"
noise_max_percent = 0.1
percentage_points = 1

In [3]:
num_steps = episode[1].shape[0] - 1
time_max = episode[1][-1]
num_qubits = episode[3].shape[0] 
if me:
    num_qubits = num_qubits - 1
    pulses = episode[2][:-1].T
else:
    pulses = episode[2].T

In [4]:
def dummy_cost_function(env):
    return 0.0

In [5]:
noise = Noise(noise_type, 0.)
env = QEnvWave(num_qubits=num_qubits,
                   time_max=time_max,
                   num_steps=num_steps,
                   cost_function=dummy_cost_function,
                   noise=noise)

In [6]:
def standard_deviation_fidelities(env, pulse, number_of_runs):
    fidelities = np.zeros(number_of_runs)
    for i in range(number_of_runs):
        fidelities[i] = env._quantum2rlstate(env.run_qevolution(pulse)).reshape((num_steps + 1, num_qubits))[-1,-1]
    return np.std(fidelities)

def number_of_runs(env, pulse, omega_percentage=0.005, number_of_runs=10):
    omeag_last = 0
    omega = standard_deviation_fidelities(env, pulse, number_of_runs)
    while omega - omeag_last > omega_percentage:
        number_of_runs *= 2
        omeag_last = omega
        omega = standard_deviation_fidelities(env, pulse, number_of_runs)
    return number_of_runs

In [7]:
def noise_percentage_effect(env, pulse, noise_max_percent):
    noise_percentages = np.linspace(0, noise_max_percent, percentage_points)
    env.noise.percentage = noise_max_percent
    print("Calculating number of runs")
    num_runs = number_of_runs(env, pulse, omega_percentage=0.001)
    results = np.zeros((len(noise_percentages), 2))
    print("Starting probing noises")
    for i, noise_percentage in enumerate(noise_percentages):
        env.noise.percentage = noise_percentage
        fidelities = np.zeros(num_runs)
        for j in range(num_runs):
            fidelities[j] = np.max(env._quantum2rlstate(env.run_qevolution(pulse)).reshape((num_steps + 1, num_qubits)).T[-1])
        results[i, 0] = np.mean(fidelities)
        results[i, 1] = np.std(fidelities)
    return results, noise_percentages

In [8]:
noise_results, noise_percentages = noise_percentage_effect(env, pulses, noise_max_percent)

Calculating number of runs
Starting probing noises
Noise percentage:  0.0
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569
10.2032
0.9775252938270569


In [ ]:
# Save results
results = [noise_results, noise_percentages]
with open("Data/3_noise_deltas_rl.pkl", "wb") as f:
    pickle.dump(results, f)